Some random sanity checks and scratchpads worth keeping around.

In [ ]:
import jax.numpy as jnp
from scipy import signal
import numpy as np
import time

from jaxdsp import processor_graph
from jaxdsp.processors import fir_filter, iir_filter, clip, delay_line, biquad_lowpass, lowpass_feedback_comb_filter as lbcf, allpass_filter, freeverb, sine_wave

## Reverb forward pass

In [ ]:
from IPython.display import Audio
from scipy.io.wavfile import read as readwav

sample_rate, X = readwav('./audio/speech-male.wav')
tail_length = 24 * sample_rate # let it ring out
X = np.concatenate([X, np.zeros(tail_length)])

In [ ]:
processor = freeverb
presets = processor.PRESETS
audio_for_preset = {preset_name: processor.tick_buffer((params, processor.init_state()), X)[1] for preset_name, params in presets.items()}

In [ ]:
# Avoid a harsh clip at the end of the sample.
def apply_release(X, release_samples=int(0.2*sample_rate)):
    return X * np.concatenate([np.ones(X.shape[-1] - release_samples), np.linspace(1.0, 0.0, release_samples)])

output_for_preset = {preset_name: Audio(apply_release(audio.T), rate=sample_rate) for preset_name, audio in audio_for_preset.items()}

In [ ]:
output_for_preset['flat_space']

In [ ]:
output_for_preset['expanding_space']

In [ ]:
import matplotlib.pyplot as plt

carry = ({"frequency_hz": 0.1}, {"sample_rate": 10.0, "phase_radians": 0.0})

X = jnp.arange(20)
carry, Y = sine_wave.tick_buffer(carry, X)
carry_2, Y_2 = sine_wave.tick_buffer(carry, X)
carry_3, Y_3 = sine_wave.tick_buffer(({"frequency_hz": 0.32}, carry_2[1]), X)
carry_4, Y_4 = sine_wave.tick_buffer(({"frequency_hz": 0.85}, carry_3[1]), X)
plt.figure(figsize=(14, 4))
plt.title("`sine_wave` phase correction over multiple frames with varying frequency", size=16)
plt.plot(jnp.concatenate([Y, Y_2, Y_3, Y_4]))
plt.vlines(np.arange(5) * X.size, ymin=-1, ymax=1, color='r', label='Frame boundaries')
_ = plt.legend()

In [ ]:
min_position = 0.0
max_position = 1.0
min_value = 30.0
max_value = 16_000.0

scale = (np.log(max_value) - np.log(min_value)) / (max_position - min_position)
position = np.linspace(min_position, max_position, 1000)
scaled = np.exp(np.log(min_value) + scale * (position - min_position))
inverse_scaled = (np.log(scaled) - np.log(min_value)) / scale + min_position

fig, [plot_1, plot_2] = plt.subplots(2, 1, figsize=(12, 6))
fig.suptitle('Exponential slider scaling', size=16)
plot_1.set_title('Exponential', size=14)
plot_1.plot(position, scaled, linewidth=2, label='exponentially scaled')
plot_1.set_ylabel('Scaled')
plot_1.hlines([min_value, max_value], xmin=min_position, xmax=max_position, color='r', linestyle='--', label='min/max values')
plot_1.legend()
plot_2.set_title('Linear & inverse (should match)', size=14)
plot_2.plot(position, position, label='linear')
plot_2.plot(position, inverse_scaled, linestyle='--', linewidth=3, label='inverse scaled')
plot_2.hlines([min_position, max_position], xmin=min_position, xmax=max_position, color='r', linestyle='--', label='min/max positions')
plot_2.set_xlabel('Position')
plot_2.set_ylabel('Scaled')
_ = plot_2.legend()
fig.tight_layout()

## TODO
* demonstrate single-sample tick
* plot processing time normalized to real_time = 1.0
* compare to C++ performance
* charts for impulse response, magnitude spectrogram and phase, updating in real-time